In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt; 


from sklearn.metrics import confusion_matrix, accuracy_score
 

from keras.utils import to_categorical #to create dummy variable
from keras.utils import np_utils
from keras.models import Sequential
from keras.applications import VGG16
from keras.applications import imagenet_utils
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.layers import Dense, Conv2D, MaxPooling2D
from keras.layers import Dropout, Flatten, GlobalAveragePooling2D
import time

import import_ipynb
import utils



from sklearn.svm import SVC

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# SOME IMPORTANT GLOBAL VARIABLE
labels = ["Bedroom","Coast","Forest","HighWay","Industrial","InsideCity","Kitchen","LivingRoom","Mountain","Office","OpenCountry","Store","Street","Suburb","TallBuilding"]
train_dir = '../images/train/'
test_dir = '../images/test/'


# CREATION OF DATA--> test and train 

As usual we, create our data

In [ ]:
#LIST OF ISTRING WHICH REPRESENT ALL THE IMAGES!
list_of_images = utils.list_of_path(labels,train_dir)
train_data,train_labels = utils.read_and_process_images(list_of_images,dimension=224)
train_labels_dummy  = to_categorical(train_labels,15)

In [ ]:
list_of_images_test = utils.list_of_path(labels,test_dir)
test_data,test_labels = utils.read_and_process_images(list_of_images_test,dimension=224)
test_labels_dummy  = to_categorical(test_labels,15)

#  model choice (VGG16) and feature extraction 

In [ ]:
model = VGG16(weights='imagenet', include_top=False)

In [ ]:
model.summary()

In [ ]:
# define a function which extract feature for all my train images

def feature_extraction(dataset,pre_model):
    """
    Function which extract feature of a dataset fro the last convolutional layer of the pre_model
    
    Input dataset= path of the train images
    Input pre_model = pre-trained model for extracting feature
    
    Output  images = images of the dataset
    Output feature = feature with shape (7,7,512)
    Output feature_flatted = flattened features 
    """
    res = []
    cont = 0
    #viaggio nel dataset
    for path in dataset:
        
        cont = cont+1
        im = load_img(path, target_size=(224, 224))
        im = img_to_array(im)
        im = np.expand_dims(im, axis=0)
        im = imagenet_utils.preprocess_input(im)
        res.append(im)
    images = np.vstack(res)
    features = pre_model.predict(images, batch_size=64)
    features_flatten = features.reshape((features.shape[0], 7 * 7 * 512))
    return images, features, features_flatten     

In [ ]:
feat_extr_time = time.time()
train_images,feature_matrix,feature_array= feature_extraction(list_of_images,model)
feat_extr_time_end = time.time()

In [ ]:
temp_for_feature = utils.convert(feat_extr_time_end-feat_extr_time)
temp_for_feature

In [ ]:
test_images,feature_matrix_test,feature_array_test = feature_extraction(list_of_images_test,model)

In [ ]:
def predict_with_linear_SVM(features_array,feature_lab,feature_array_test,test_labels,num_classes = 15):
    """
    Function which exploits linear svm to predict classes of test data 
    
    Input features_array = array with train features (flattened)
    Input feature_lab = labels of train features
    Input feature_array_test = array with test features (flattened)
    Input test_labels = labels of test features
    Input num_classes (15) = number of classes
    
    Output classif = array of svm classidicator
    Output prediction = array of prediction of test features
    Output acc = accuracy of prediction 
    """
    
    classif = [SVC(kernel="linear") for _ in range(num_classes)] # array of classifier to be trained
    # one-vs-all approach with  
    curr_label = 0
    for clf in classif:
        v = np.array([1 if label==curr_label else 0 for label in feature_lab])
        clf = clf.fit(features_array, v)
        curr_label = curr_label + 1
    # now we want to test 
    prediction = []
    for image in feature_array_test:
        pred = np.array([np.dot(clf.coef_,image) + clf.intercept_ for clf in classif])
        prediction.append(np.argmax(pred))
    prediction = np.asarray(prediction)
    #calculate accuracy
    cont=0
    for i in range(len(prediction)):
        if prediction[i]==test_labels[i]:
            cont = cont +1
        else:
            continue
        
    acc = cont/len(prediction)
    
    return classif,prediction,acc
                    
    
    

In [ ]:
start_svm = time.time()
classifiers,prediction,accuracy = predict_with_linear_SVM(feature_array,train_labels,feature_array_test,test_labels)
end_evm = time.time()

In [ ]:
T = utils.convert(end_evm-start_svm)
T

In [ ]:
accuracy

In [ ]:
def evaluated_prediction(pred,test_lab,num_classes = 15,lab = labels):
    """
    Function which evaluate quality of prdiction of linear svm, calculating TP,FP,FN,TN
    
    Input pred = prediction 
    Input test_lab = labels
    Input num_classes(15) = number of classes
    Input lab = name of classes
    
    Output  res = dataframe with all these values
    """
    tp = []
    fp = []
    fn = []
    tn = []
    for i in range(num_classes):
        tp_temp = 0
        fp_temp = 0
        fn_temp = 0
        tn_temp = 0
        for j in range(len(pred)):
            if(pred[j]==i and test_lab[j]==i):
                tp_temp = tp_temp + 1
            if(pred[j]==i and test_lab[j]!=i):
                fp_temp = fp_temp + 1
            if(pred[j]!=i and test_lab[j]==i):
                fn_temp = fn_temp + 1
            if(pred[j]!=i and test_lab[j]!=i):
                tn_temp = tn_temp +1
        tp.append(tp_temp)
        fp.append(fp_temp)
        fn.append(fn_temp)
        tn.append(tn_temp)
    data = {'labels':labels , 'True positive':tp,'True negative':tn,'False positive':fp,'False negative':fn}
    res = pd.DataFrame(data, columns = ['labels','True positive','True negative','False positive','False negative'])
    return res

In [ ]:
prova = evaluated_prediction(prediction,test_labels)

In [ ]:
prova

In [ ]:
prova.loc[4]['labels']

In [ ]:
def build_confusion_matrix(df,pred,test_labels,lab = labels):
    """
    Function tu construct confusion matrix
    """
    num_classes = len(lab)
    cm = np.zeros((15,15))
    # insert true positive on the diagonal
    for i in range(num_classes):
        cm[i,i] = df.loc[i]['True positive']
    for i in range(num_classes): # lavoro sulle classes true
        for j in range(num_classes): #lavoro su classes predicted 
            temp = 0
            for k in range(len(test_labels)):
                if(test_labels[k]==i and pred[k]==j):
                    temp = temp +1
            cm[i,j]=temp
    return cm   
    

In [ ]:
d = build_confusion_matrix(prova,prediction,test_labels)

In [ ]:
plt.figure(figsize=(12,6))
vvv = utils.plot_confusion_matrix(d,labels,"point_three_linear_SVM","images_point_three/",normalize=True)